Dear User,

This is a tutorial script showcasing the testing modules of the IonoBench Framework. It also serves as an introductory documentation script for the framework for you to download datasets and trained models.

**Instructions** 

You can download the provided dataset and pretrained models, then run the test modules to replicate paper results.

This notebook is designed for **Google Colab** and requires a **GPU runtime**.
To enable GPU: click on the `down arrow` **right** beside `Connect` **in the top-right**, then go to `Change runtime type > Hardware accelerator > GPU (T4 GPU etc.)`.

Please run the notebook **step-by-step**, following the comments provided.

There are three main testing sections:
- **3b. Default Test**  
- **4a. Solar Analysis**  
- **4b. Storm Analysis**

Both the default test and solar analysis are commented out for your convenience, as each may take ~20 minutes to run (on Colab GPU). If you prefer a quicker test, you can leave them commented and run **Storm Analysis** section, which completes in a few minutes.

After that section you can observe the paper results on `5.3.3. Visual Comparison: Residual Patterns during Stormy vs. Quiet Conditions` (Specifically first two rows: stormy example.)
If you wait for other test runs you can reproduce the paper results `5.1. Overall Performance` and `5.2. Performance Across Solar Activity Levels`

Start with the default model “SimVPv2.”
If you want to **test other models**, just restart the notebook. Make sure you;
- Download the new model weights from `2: Models and Configs` and change the path on `3a: Loading Pre-trained Model`.
- Change the model name in the configs.
- Give each run a unique session name so the results are stored separately. before you start the test parts.

**Common Issues You Might Face**
- **Session crash:** On the upper left panel find `Runtime` and select `Restart Session`. (Good news: In the second run you won't need to wait for downloads)
- **NVIDIA driver error:** Check if the GPU is correctly shown in the `# 0: Preps >>> Verify GPU session`.
- **Timeout:** Colab sessions are limited; you may need to restart and rerun.
- **Download errors:** If dataset or model download fails due to interruption, delete any existing `datasets/` or `training_sessions/` folders from the loaded git folder and try again.

**Note: All repeating cells used for data preparation before different testing types will be hidden using CLI support. Currently this notebook is showing how the underlying config structure and functions works.**

--- 
## 0: Preps  
Clone repo, install requirements, and verify GPU


In [ ]:
# 0: Preps >>> Verify GPU session
#================================================================
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
print(f'CUDA version: {torch.version.cuda}')
#================================================================

Please don't continue if `CUDA available: False` and select GPU again and restart session.

--- 
#### Colab Preps (Skip if local build)

In [ ]:
# 0: Preps >>> Clone Repo (Skip here if you are running locally)
#================================================================
!git clone https://github.com/Mert-chan/IonoBench --quiet
%cd IonoBench
#================================================================

In [ ]:
# 0: Preps >>> Download the required libs
#================================================================
!pip install -r /content/IonoBench/requirements_colab.txt --quiet              # Can take couple of minutes
!pip install --quiet torch torchvision --index-url https://download.pytorch.org/whl/cu118 # Install cu118 Just incase if pytorch can't find 
#================================================================

--- 
## 1: Dataset  


If chronological split is desired, change to "chronological" However for chronological you can only replicate the Paper Results on  `Section 4 Investigating Future Bias in Stratified Split `

In [ ]:
# 1: Dataset >>> Download the desired IonoBench dataset (Stratified or Chronological split)
#==================================================================================
from pathlib import Path
import os,sys

repo_name = "IonoBench"  
base_path = Path(f"/content/{repo_name}")
sys.path.append(str(base_path))
sys.path.append('./source')
sys.path.append('./scripts')
from source.myDataFuns import download_dataset
download_dataset(
    dataset_name="stratified",  # If chronological split is desired, change to "chronological" However for chronological you can only replicate the Paper Results on Section 4
    base_path=base_path
    )     
#==================================================================================


In [ ]:
# 1: Dataset >>> Read dataset
#====================================================================================
from scripts.data import load_training_data

dataDict = load_training_data(
                            seq_len=12,                
                            pred_horz=12,
                            datasplit='stratified',      # Need to be changed according to the datasplit
                            features = None,             # Default "None" loads all features, otherwise specify a list of features i.e ['F10.7', 'Dst']
                            base_path=base_path          # You can also see the full list of features in ~/configs/base.yaml
                            )        


print("\n", "-" * 20, "\n", dataDict.keys())
print("You can use the dataDict following dict_keys as you like from this point on.")
#====================================================================================

In [ ]:
# 1: Dataset >>> Ex: Accessing specific OMNI features
#====================================================================================
dataDict['OMNI_names']
#=====================================================================================

In [ ]:
# 1: Dataset >>> Ex: Accessing specific OMNI features (note: all features are normalized to 0-1) 
#=====================================================================================
'''
"normOMNI" is same order with the "OMNI_names"
Check the name and access the desired feature using the getOMNIfeature function.
'''
from source.myDataFuns import getOMNIfeature

# To access 'Dst' values 
print("Dst:", getOMNIfeature(dataDict,"Dst")) # Dst values
print("Year:", getOMNIfeature(dataDict,"Year")) # Year values

#======================================================================================

In [ ]:
# 1: Dataset >>> Ex: Accessing TEC data for specific date range (note: tec data is normalized to 0-1)
#======================================================================================
from datetime import datetime, timedelta
from source.myDataFuns import dateList
# Select the date period
startDate = datetime(2024, 5, 9, 0, 0)  # Start date
endDate = datetime(2024, 5, 9, 22, 0)    # End date (inclusive)

# Find idx corresponding to start and end dates
startidx = dataDict['dates'].index(startDate)
endidx = dataDict['dates'].index(endDate)

# Extract TEC data for the selected date range
norm_tecData = dataDict['normTEC'][startidx:endidx+1]
date_list = dateList(startDate, endDate,timedelta(hours=2)) # TEC maps are 2 hours apart
print("TEC data shape for the selected date range:", norm_tecData.shape)
print("Dates of the selected date range:", len(date_list))
#======================================================================================

In [ ]:
# 1: Dataset >>> Reversing the Preprocessing Steps and Visualizing the Original TEC Data
#======================================================================================
'''
To recover original TEC data, the preprocessing steps (Normalization and heliocentric transformation) needed to be reversed. 
'''
from source.myDataFuns import reverseHeliocentric
tecData = norm_tecData*(dataDict['maxTEC'] - dataDict['minTEC']) + dataDict['minTEC']  # Reverse normalization
org_tecData = reverseHeliocentric(tecData, date_list)

from source.myVisualFuns import makeComparison_Anim
from IPython.display import HTML
# Define the plot titles for the animation
plot_titles = {
    'main': 'Reverse Heliocentric Transformation',
    'subplot1': 'Real VTEC IGS',
    'subplot2': 'Longitude-Shifted VTEC IGS',
    'colorbar': 'TECU (10$^{16}$ e/m$^2$)'
}

# Create a comparison animation of the original and transformed TEC data
'''
This is for demonstration purposes. Reversing the effect of the heliocentric transformation is shown in below animation
Right side shows after reverse heliocentric transformation, left side shows dataset TEC data before transformation.
'''
anim = makeComparison_Anim(
    data1=org_tecData,
    data2=tecData,
    date_list=date_list,
    titles=plot_titles,
    show_metrics=False
)
HTML(anim.to_jshtml())  
#======================================================================================

--- 
## 2: Models  

In [ ]:
# 2: Models and Configs >>> Download the desired Trained Model (SimVP2,SwinLSTM, DCNN etc.) from Hugging Face Hub
#==================================================================================
from pathlib import Path

from source.myDataFuns import download_model_folder

data_path = Path(base_path, "training_sessions")         # ..~/DemoRepo/training_sessions 
data_path.mkdir(parents=True, exist_ok=True)             # Create new folder when model folder is downloaded.

                          #
download_model_folder(                                   # This function automatically downloads the paper's model folder from Hugging Face Hub.
                    model_name = "SimVPv2",              # Change to "DCNN", "SwinLSTM", "SimVPv2_Chrono" for other models.
                    base_path = base_path
                    )    
#==================================================================================


In [ ]:
# 2: Models and Configs >>> Load the model configurations
#==================================================================================
from scripts.loadConfigs import load_configs

'''
Configs uses base => model => mode => CLI override hierarchy.
You can change the model and mode to load different configurations.
For example, change "SimVPv2" to "DCNN121", "SwinLSTM" etc. to load different model configurations.
load_configs will return a merged configurations of the base, model, and mode.
'''

cfgs = load_configs(
                 model = "SimVPv2",  # Change to "DCNN121", "SwinLSTM", "SimVPv2_Chrono" for other models.
                 mode = "test",         
                 split = "stratified",  
                 base_path= base_path
                 )
#==================================================================================

In [ ]:
# 2: Models and Configs >>> Can check the keys of the config dictionary
#================================================================
print(cfgs.keys())
print(cfgs.data.keys())
print(cfgs.model.keys())
print(cfgs.data.data_split)
print(cfgs.model)
#================================================================

--- 
## 3: Testing Trained Models 
--- 


### 3a: Loading Pre-trained Model


In [ ]:
# 3a: Loading Pre-trained Model>>> Prepare the data and build the model     
#=================================================================

from scripts.registry import build_model
from scripts.data import prepare_raw

# Prepare the data for testing
data = prepare_raw(cfgs)               # This fun Wraps load_training_data & patches cfg with shape/min-max info.
B = cfgs.test.batch_size               # batch size from YAML
T = cfgs.data.seq_len                  # input sequence length
C = cfgs.data.num_omni + 1             # OMNI scalars + TEC map channel
H = cfgs.data.H                        # height (set inside prepare_raw)
W = cfgs.data.W                        # width  (set inside prepare_raw)
cfgs.test.input_names = data['OMNI_names']          # This sets the input names for test logging file


device = "cuda:0" if torch.cuda.is_available() else "cpu"  # Use GPU if available, otherwise CPU
# Build the model
cfgs.model.input_shape = (T, C, H, W)  # Set input shape for the model
model = build_model(cfg = cfgs, base_path=base_path, device=device)           # Build the model with the configurations
print(f"Batch size: {B}, T: {T}, C: {C}, H: {H}, W: {W}")
#==================================================================

In [ ]:
# 3a: Loading Pre-trained Model >>> Summary of the model   
#=================================================================
from torchinfo import summary
summary(model, input_size=((B, C, T, H, W),(B, cfgs.data.pred_horz, H, W)))
#=================================================================


!!!! **Before loading the weights to build model**  Change the Path of **torch.load** to the path of your downloaded model checkpoint folder if you are trying DCNN or SwinLSTM. (Default is set to SimVPv2)
You can find the checkpoint file in the downloaded model folder under `training_sessions/{modelName}` as .... `"NameofTheSession"_best_checkpoint_"yyyymmdd"_"hhmm" `
To download new pre-trained model, refer back to the `2: Models and Configs >>> Download the desired Trained Model` section.


In [ ]:
# 3a: Loading Pre-trained Model >>> Load the model weights from a checkpoint file
#====================================================================================
from source.myTrainFuns import DDPtoSingleGPU
import torch

checkpoint = torch.load(
    r'/content/IonoBench/training_sessions/SimVPv2/SimVP_stratifiedSplit_Allfeatures_best_checkpoint_20250320_1401.pth',            # <= Copy inside
    weights_only=True)
model.load_state_dict(DDPtoSingleGPU(checkpoint["model_state_dict"]))   # Load the model state dict If DDP(Multiple GPU) was used get rid of Module prefix
#====================================================================================

--- 
### 3b: Default Test

---

**!!! Disclaimer !!!**  
The paper’s experiments were primarily trained and tested using **4 GPUs with Distributed Data Parallel (DDP)**.  
DDP introduces some non-determinism, even with fixed random seeds, due to differences in data shuffling and the order of floating-point operations across GPUs.

This notebook uses a **single GPU**, which may lead to minor variations between the metrics generated here and those reported in the paper

These differences are **statistically negligible** and can be validated by comparing with the reported results.  
The key point is that these differences will not affect the paper's overall findings or the relative performance ranking of the models.

For bit-for-bit reproducibility of your own experiments, it is essential to use a fixed hardware configuration (e.g., the same number of GPUs) for both training and evaluation.

---

In [ ]:
# 3b: Default Test >>> Setup the data loaders for testing
#================================================================
from scripts.data import make_default_loaders
loaders = make_default_loaders(cfg = cfgs, d = data)                # Make default loaders for training, validation, and test sets.
len(loaders["train"]), len(loaders["valid"]), len(loaders["test"])
#================================================================

In [ ]:
# 3b: Default Test >>> Test the model (Default: test set) (RUN TIME: >~20 min)
#================================================================
from source.myTrainFuns import IonoTester

# FOR DEFAULT TESTING UNCOMMENT BELOW
#================================================================
'''
cfgs.test.save_results = True           # Save the test results to a log file (txt).
cfgs.test.save_raw = False              # Save the raw test results (Prediction Dates, TEC predictions and dedicated truth maps) to npz.
cfgs.session.name = "SimVPv2_test"      # Write a session name for the test results. New name will create a new folder in the ~/IonoBenchv1/training_sessions/
testDict = IonoTester(model, loaders['test'], device=device, config=cfgs).test() 
'''
#====================================================================================

--- 
## 4: Solar and Storm Analysis
---
  


### 4a: Solar Analysis

In [ ]:
# 4a: Solar and Storm Analysis >>> Solar Loaders 
#====================================================================================
from scripts.data import make_solar_loaders

if "loaders" in locals():
    del loaders, data, cfgs             # To clear memory before loading solar loaders
    
import gc
gc.collect()
torch.cuda.empty_cache()

cfgs = load_configs(
                 model = "SimVPv2",  # <= change to "DCNN121", "SwinLSTM" for other models. (But the loaded model should match the model name)
                 mode = "solar",     # <= The testing type is set to "solar" to load solar configurations dynmaically.
                 split = "stratified",  
                 base_path= base_path
                 )
cfgs.paths.base_dir = base_path
data = prepare_raw(cfgs)                # Prepare the data (again neeeded for setting important parameters to config inside) 
loaders = make_solar_loaders(cfg = cfgs, base_path=base_path, d = data)                # Make default loaders for training, validation, and test sets.
#====================================================================================

In [ ]:
# 4a: Solar Analysis >>> Analysis Function (Testing on Solar intensity classes: very weak, weak, moderate, intense) (RUN TIME: >~20 Mins)
#====================================================================================
from source.myTrainFuns import SolarAnalysis

# FOR SOLAR ANALYSIS UNCOMMENT BELOW
#================================================================
'''
cfgs.session.name = "SimVPv2_test"  # Writes on top of the previous test file if the session name is the same.
cfgs.test.save_results = True       # Appends the test results to a log file (txt) that exists in the training_sessions folder under the session name. (if not exists, creates a new one) 
cfgs.test.save_raw = False          # Save the raw test results (Prediction Dates, TEC predictions and dedicated truth maps) to npz. (Per solar class)
solarDict = SolarAnalysis(model, data, loaders, device=device, cfg=cfgs).run()
'''
#====================================================================================

---
### 4b: Storm Analysis

In [ ]:
# 4b: Storm Analysis >>> Storm Loaders
#====================================================================================
from scripts.data import make_storm_loaders

if "loaders" in locals():
    del loaders, data, cfgs             # To clear memory before loading solar loaders
    
import gc
gc.collect()
torch.cuda.empty_cache()

cfgs = load_configs(
                 model = "SimVPv2",  # <= change to "DCNN121", "SwinLSTM" for other models. (But the loaded model should match the model name)
                 mode = "storm",     # <= changed to storm to load storm configurations dynamically.
                 split = "stratified", 
                 base_path= base_path
                 )
cfgs.paths.base_dir = base_path
data = prepare_raw(cfgs)                # Prepare the data (again neeeded for setting important parameters to config inside) 
loaders = make_storm_loaders(cfg = cfgs, d = data)                # Make default loaders for training, validation, and test sets.
#====================================================================================

In [ ]:
# 4b: Storm Analysis >>> Analysis Function (Testing on Storm events) (RUN TIME: ~1-3 Mins)
#====================================================================================
from source.myTrainFuns import StormAnalysis

cfgs.test.save_results = True           # Save the test results to a log file (txt).
cfgs.test.save_raw = True               # Save the raw test results (Prediction Dates, TEC predictions and dedicated truth maps) to npz.
cfgs.session.name = "SimVPv2_test"      # Write a session name for the test results. New name will create a new folder in the ~/IonoBenchv1/training_sessions/
testDict = StormAnalysis(model, data, cfgs, loaders,device).run() 
#====================================================================================

--- 
## 5: Storm Predictions and Residuals
---
  

In [ ]:
from IPython.display import HTML
import numpy as np
from source.myVisualFuns import spatialComparison_Anim

# Example Storm Animation (Storm 2, 2001-11-06)
npz_path = base_path / "training_sessions" / cfgs.session.name / "storms" / "test_raw_storm_2_2001-11-06.npz"       # You can change visualize all storms by changing npz_path to the desired storm npz file
# Note: npz_path can be found in the ~/IonoBenchv1/training_sessions/SimVPv2_test/storms/ folder after running the StormAnalysis function above.

anim = spatialComparison_Anim(
    npz_path=npz_path,
    dataDict=dataDict,
    cfgs=cfgs,
    n_start=33,             # Starting storm index (0-72, default 0) 33 is start of main phase +-6h
    n_end=40,               # Ending storm index (0-72, default None, uses all) 40 is end of main phase +-6h
    horizon=6,              # 0-11 steps: 0 → 2h, 5 → 12h, etc.
    interval_ms=300,
    save=True              # set False if you only want inline display
)

HTML(anim.to_jshtml())
